In [ ]:
import sys
sys.path.append("../../")
import warnings
import numpy as np
import astropy.units as u
from astropy.constants import h, e, m_e, c, sigma_T
from agnpy.synchrotron import R, epsilon_equivalency
from agnpy.compton import compton_kernel
from agnpy.emission_regions import Blob
from agnpy.targets import PointSourceBehindJet
import matplotlib.pyplot as plt

e = e.gauss

## my implementation of the trapezoidal rule in log-log space

In [ ]:
def log(x):
    # smallest positive float (before 0)
    float_tiny = np.finfo(np.float64).tiny
    # largest positive float
    float_max = np.finfo(np.float64).max
    values = np.clip(x, float_tiny, float_max)
    return np.log(values)

In [ ]:
def trapz_loglog(y, x, axis=0):
    """
    Integrate along the given axis using the composite trapezoidal rule in
    loglog space.

    Integrate `y` (`x`) along given axis in loglog space.

    Parameters
    ----------
    y : array_like
        Input array to integrate.
    x : array_like, optional
        Independent variable to integrate over.
    axis : int, optional
        Specify the axis.

    Returns
    -------
    trapz : float
        Definite integral as approximated by trapezoidal rule in loglog space.
    """
    try:
        y_unit = y.unit
        y = y.value
    except AttributeError:
        y_unit = 1.0
    try:
        x_unit = x.unit
        x = x.value
    except AttributeError:
        x_unit = 1.0
    
    slice_low = [slice(None)] * y.ndim
    slice_up = [slice(None)] * y.ndim
    # multi-dimensional equivalent of x_low = x[:-1]
    slice_low[axis] = slice(None, -1)
    # multi-dimensional equivalent of x_up = x[1:]
    slice_up[axis] = slice(1, None)

    slice_low = tuple(slice_low)
    slice_up = tuple(slice_up)

    # reshape x to be broadcasted with y
    if x.ndim == 1:
        shape = [1] * y.ndim
        shape[axis] = x.shape[0]
        x = x.reshape(shape)
        
    x_low = x[slice_low] 
    x_up = x[slice_up]
    y_low = y[slice_low]
    y_up = y[slice_up]
    
    log_x_low = log(x_low)
    log_x_up = log(x_up)
    log_y_low = log(y_low)
    log_y_up = log(y_up)
    
    # index in the bin
    m = (log_y_low - log_y_up) / (log_x_low - log_x_up)
    vals = y_low / (m + 1) * (x_up * (x_up / x_low) ** m - x_low)
    
    tozero = (y_low == 0.0) + (y_up == 0.0) + (x_low == x_up)
    vals[tozero] = 0.0
    
    
    return np.add.reduce(vals, axis) * x_unit * y_unit

## a simple test with a straight line in log-log scale

In [ ]:
def line_loglog(x, m, n):
    """a straight line in loglog-space"""
    return x ** m * np.e ** n


def integral_line_loglog(x_min, x_max, m, n):
    """analytical integral of the line in log-log space"""
    f_low = line_loglog(x_min, m + 1, n) / (m + 1)
    f_up = line_loglog(x_max, m + 1, n) / (m + 1)
    return f_up - f_low

In [ ]:
m = 1.5
n = -2.0
x = np.logspace(2, 5)
y = line_loglog(x, m, n)
y = np.asarray([y, y])

In [ ]:
trapz_loglog(y.T, x, axis=0)

In [ ]:
integral_line_loglog(x[0], x[-1], m, n)

In [ ]:
np.trapz(y.T, x, axis=0)

In [ ]:
1 - trapz_loglog(y.T, x, axis=0) / integral_line_loglog(x[0], x[-1], m, n)

In [ ]:
1 - np.trapz(y.T, x, axis=0) / integral_line_loglog(x[0], x[-1], m, n)

## a test with synchrotron radiation

In [ ]:
blob = Blob()
nu = np.logspace(9, 20, 20) * u.Hz

# check the blob
print(blob)

In [ ]:
def sed_synch(nu, integration):
    """compute the synchrotron SED"""
    epsilon = nu.to("", equivalencies=epsilon_equivalency)
    # correct epsilon to the jet comoving frame
    epsilon_prime = (1 + blob.z) * epsilon / blob.delta_D
    # electrond distribution lorentz factor
    gamma = blob.gamma
    N_e = blob.N_e(gamma)
    prefactor = np.sqrt(3) * epsilon * np.power(e, 3) * blob.B_cgs / h
    # for multidimensional integration
    # axis 0: electrons gamma
    # axis 1: photons epsilon
    # arrays starting with _ are multidimensional and used for integration
    _gamma = np.reshape(gamma, (gamma.size, 1))
    _N_e = np.reshape(N_e, (N_e.size, 1))
    _epsilon = np.reshape(epsilon, (1, epsilon.size))
    x_num = 4 * np.pi * _epsilon * np.power(m_e, 2) * np.power(c, 3)
    x_denom = 3 * e * blob.B_cgs * h * np.power(_gamma, 2)
    x = (x_num / x_denom).to_value("")
    integrand = _N_e * R(x)
    integral = integration(integrand, gamma, axis=0)
    emissivity = (prefactor * integral).to("erg s-1")
    sed_conversion = np.power(blob.delta_D, 4) / (
        4 * np.pi * np.power(blob.d_L, 2)
    )
    return (sed_conversion * emissivity).to("erg cm-2 s-1")

In [ ]:
%%timeit
sed_synch(nu, np.trapz)

In [ ]:
%%timeit
sed_synch(nu, trapz_loglog)

In [ ]:
sed_trapz = sed_synch(nu, np.trapz)
sed_trapz_loglog = sed_synch(nu, trapz_loglog)
plt.loglog(nu, sed_trapz, marker="o")
plt.loglog(nu, sed_trapz_loglog, ls="--", marker=".")
plt.show()

## a test with inverse Compton radiation

In [ ]:
def sed_flux_point_source(nu, target, r, integrate):
    """SED flux for EC on a point like source behind the jet

    Parameters
    ----------
    nu : `~astropy.units.Quantity`
        array of frequencies, in Hz, to compute the sed, **note** these are 
        observed frequencies (observer frame).
    """
    # define the dimensionless energy
    epsilon_s = nu.to("", equivalencies=epsilon_equivalency)
    # transform to BH frame
    epsilon_s *= 1 + blob.z
    # for multidimensional integration
    # axis 0: gamma
    # axis 1: epsilon_s
    # arrays starting with _ are multidimensional and used for integration
    gamma = blob.gamma_to_integrate
    transformed_N_e = blob.N_e(gamma / blob.delta_D).value
    _gamma = np.reshape(gamma, (gamma.size, 1))
    _N_e = np.reshape(transformed_N_e, (transformed_N_e.size, 1))
    _epsilon_s = np.reshape(epsilon_s, (1, epsilon_s.size))
    # define integrating function
    # notice once the value of mu = 1, phi can assume any value, we put 0
    # convenience
    _kernel = compton_kernel(
        _gamma, _epsilon_s, target.epsilon_0, blob.mu_s, 1, 0
    )
    _integrand = np.power(_gamma, -2) * _N_e * _kernel
    integral_gamma = integrate(_integrand, gamma, axis=0)
    prefactor_num = (
        3
        * sigma_T
        * target.L_0
        * np.power(epsilon_s, 2)
        * np.power(blob.delta_D, 3)
    )
    prefactor_denom = (
        np.power(2, 7)
        * np.power(np.pi, 2)
        * np.power(blob.d_L, 2)
        * np.power(r, 2)
        * np.power(target.epsilon_0, 2)
    )
    sed = prefactor_num / prefactor_denom * integral_gamma
    return sed.to("erg cm-2 s-1")

In [ ]:
# target and distance
r = 1e16 * u.cm 
L_0 = 2e46 * u.Unit("erg s-1")
epsilon_0 = 1e-3
ps = PointSourceBehindJet(L_0, epsilon_0)

nu = np.logspace(20, 30) * u.Hz

# increase the size of the gamma grid
blob.set_gamma_size(1000)

In [ ]:
%%timeit
sed_flux_point_source(nu, ps, r, np.trapz)

In [ ]:
%%timeit
sed_flux_point_source(nu, ps, r, trapz_loglog)

In [ ]:
sed_trapz = sed_flux_point_source(nu, ps, r, np.trapz)
sed_trapz_loglog = sed_flux_point_source(nu, ps, r, trapz_loglog)
plt.loglog(nu, sed_trapz, marker="o")
plt.loglog(nu, sed_trapz_loglog, ls="--", marker=".")
plt.show()

In [ ]:
sed_trapz_loglog